<a href="https://colab.research.google.com/gist/river-li/2a2a3f9d692be95d5a331df556444b67/mnist_interval_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Interval Analysis

In [1]:
!pip install tensorboardX torch matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.6 MB/s eta 0:00:00


In [11]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Sequential):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = (x - 0.1307)/0.3081
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=-1) # added softmax for probabilities
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
# model = nn.Sequential(Normalize(), Net())

model = Net()
model = model.to(device)
model.train()


Net(
  (fc): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=10, bias=True)
)

In [12]:
def train_model(model, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.3f}')

def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy on images: {100 * correct / total}')

In [13]:
train_model(model, 15)
test_model(model)

Epoch 1/15, Loss: 2.012
Epoch 2/15, Loss: 1.729
Epoch 3/15, Loss: 1.627
Epoch 4/15, Loss: 1.596
Epoch 5/15, Loss: 1.582
Epoch 6/15, Loss: 1.572
Epoch 7/15, Loss: 1.566
Epoch 8/15, Loss: 1.561
Epoch 9/15, Loss: 1.557
Epoch 10/15, Loss: 1.553
Epoch 11/15, Loss: 1.550
Epoch 12/15, Loss: 1.547
Epoch 13/15, Loss: 1.545
Epoch 14/15, Loss: 1.542
Epoch 15/15, Loss: 1.540
Accuracy on images: 93.43


### Write the interval analysis for the simple model

In [14]:
## TODO: Write the interval analysis for the simple model
## you can use https://github.com/Zinoex/bound_propagation
!pip install bound-propagation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.1 MB/s eta 0:00:00


In [15]:
from bound_propagation import BoundModelFactory, HyperRectangle

factory = BoundModelFactory()
bounded_net = factory.build(model)

def verify(bounded_net, images, labels, epsilon):
    verified =0
    total = 0
    for i in range(len(images)):
      x = images[i:i+1].view(1, -1)
      lower = torch.clamp(x - epsilon, min=0)
      upper = torch.clamp(x + epsilon, max=1)
      input_bounds = HyperRectangle(lower, upper)
      output_bounds = bounded_net.ibp(input_bounds)
      # output_bounds = bounded_net.crown_ibp(input_bounds)

      label = labels[i].item()
      true_lower = output_bounds.lower[0, label]
      is_robust = True
      for j in range(10):
          if j != label and true_lower <= output_bounds.upper[0, j]:
              # compare all other categories's upper with the true category's lower
              is_robust = False
              break
      if is_robust:
          verified += 1
      total += 1
    return verified / total * 100


In [16]:
eps_range = np.linspace(0.01, 0.1, 10)

for epsilon in eps_range:
    images, labels = next(iter(test_loader))
    accuracy = verify(bounded_net, images, labels, epsilon)
    print(f"ε = {epsilon:.3f}: {accuracy:.1f}% verified")

ε = 0.010: 14.1% verified
ε = 0.020: 1.6% verified
ε = 0.030: 0.0% verified
ε = 0.040: 0.0% verified
ε = 0.050: 0.0% verified
ε = 0.060: 0.0% verified
ε = 0.070: 0.0% verified
ε = 0.080: 0.0% verified
ε = 0.090: 0.0% verified
ε = 0.100: 0.0% verified
